# Binary String Classfication

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.contrib.rnn import RNNCell, BasicLSTMCell, MultiRNNCell, DropoutWrapper

## Creating Interactive Session

In [2]:
init_op = tf.global_variables_initializer()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.001, allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(init_op)

## Data

In [3]:
LABEL_SIZE = 100
TRAINING_SIZE = 50000
TEST_SIZE = 10000

def create_data(size=50000, maximum=1000, end_token=2):
    max_length = len(bin(maximum-1)[2:]) + 1
    rands = np.random.randint(0, maximum, size=500000)
    xs = np.zeros((size, 1, max_length), dtype='float32')
    ys = np.zeros((size, 1, maximum), dtype='int32')
    for i in range(size):
        x = np.zeros(20, dtype='float32')
        rand_bin = bin(rands[i])[2:]+ str(end_token)
        rand_bin = list(map(float, list(rand_bin)))
        xs[i, 0, :len(rand_bin)] = rand_bin    
        ys[i, 0, rands[i]] = 1
    
    return xs, ys

train_x, train_y = create_data(size=TRAINING_SIZE, maximum=LABEL_SIZE)
test_x, test_y = create_data(size=TEST_SIZE, maximum=LABEL_SIZE)

SENTENCE_LENGTH = train_x.shape[2]

In [4]:
def next_batch(x, y, size=50):
    idx = np.random.randint(x.shape[0] - size)
    return x[idx:idx+size], y[idx:idx+size]

sample_x, sample_y = next_batch(train_x, train_y)
print(sample_x.shape, sample_y.shape)

(50, 1, 8) (50, 1, 100)


## LSTM Model

In [5]:
hidden_size = 5

with tf.variable_scope('test' + str(np.random.randint(0, 100000))):
    inputs = tf.placeholder('float32', shape=[None, None, SENTENCE_LENGTH], name='inputs') # [batch, time, in]
    targets = tf.placeholder('float32', shape=[None, None, LABEL_SIZE], name='targets') # [batch, time, out]
    
    cell = BasicLSTMCell(LABEL_SIZE, forget_bias=1.0, state_is_tuple=True)
#     cell = DropoutWrapper(cell)
    init_state = cell.zero_state(1, 'float32')

    rnn_outputs, rnn_states = tf.nn.dynamic_rnn(cell, inputs, initial_state=init_state, time_major=True)
#     rnn_outputs = tf.reshape(rnn_outputs, [-1, hidden_size])
    
#     w = tf.get_variable('weights', [hidden_size, 1], initializer=tf.random_normal_initializer())
#     b = tf.get_variable('biases', [1, 1], initializer=tf.constant_initializer())
    
#     dense1 = tf.matmul(rnn_outputs, w) + b
#     dense2 = tf.reduce_sum(dense1, reduction_indices=[1])
#     prediction = tf.nn.sigmoid(dense2)
    
    error2 = tf.square(targets - rnn_outputs)
    error3 = tf.reduce_mean(error2, reduction_indices=[0]) 
    train_fn = tf.train.AdamOptimizer(learning_rate=0.01).minimize(error3)
    
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [6]:
def train(x, y, n_epoch=5, batch_size=50):
    
    for epoch in range(n_epoch):
        costs = []
        for step in range(int(x.shape[0]/batch_size)):
            sample_x, sample_y = next_batch(x, y, size=batch_size)
            cost, _ = sess.run([error3, train_fn], 
                                   feed_dict={inputs: sample_x, targets: sample_y})
            costs.append(np.sum(cost))

    #         print('xxxxx', cost.shape)
    #         print(cost)
    #         break

        cost = sum(costs)/float(step + 1)
        print(f'[{epoch}] cost: {cost}')

train(train_x, train_y)        

[0] cost: 0.6340201211571693
[1] cost: 0.30096231620013714
[2] cost: 0.16398536373674868
[3] cost: 0.10415067917108536
[4] cost: 0.07905111364275218


In [7]:
def evaluate(x, y):
    test_size = x.shape[0]
    DATA_LIMIT = 200
    
    n_correct = 0
    global_step = 0
    for i in range(0, test_size, DATA_LIMIT):
        y_preds = sess.run(rnn_outputs, feed_dict={inputs: x[i:i+DATA_LIMIT]})

        for j in range(DATA_LIMIT):
            y_pred = np.argmax(y_preds[j, 0])
            y_true = np.argmax(y[j, 0])
            if y_pred == y_true:
                n_correct += 1
                
            global_step += 1
    
#     n_correct = 0
#     for i in range(test_size):
#         y_pred = np.argmax(y_preds[i, 0])
#         y_true = np.argmax(y[i, 0])
#         if y_pred == y_true:
#             n_correct += 1
            
    print(f'테스트 갯수: {test_size}, 맞은 갯수: {n_correct}')
    print('accuracy:', n_correct/float(test_size))

evaluate(test_x, test_y)

테스트 갯수: 10000, 맞은 갯수: 288
accuracy: 0.0288


In [17]:
test_x.shape

(10000, 1, 8)